# MVP of setting up a mulit-agent LLM chain

Specifically for the financial sector

# Install & Import Depediencies

In [ ]:
import os
import sys
import asyncio
from typing import List, Dict, Any

In [4]:
# Ensure necessary imports are available
#%%capture --no-stderr
%pip install --quiet -U langchain 
%pip install --quiet -U langgraph 
%pip install --quiet -U openai 
%pip install --quiet -U python-dotenv

Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


Note: you may need to restart the kernel to use updated packages.


Could not find platform independent libraries <prefix>


In [6]:
import dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint import MemorySaver

# Load environment variables
dotenv.load_dotenv()


ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
# 
class StockMarketAgentState:
    """Defines the state for the multi-agent workflow."""
    def __init__(self, question: str):
        self.question = question
        self.expert_analyses: List[Dict[str, str]] = []
        self.consensus_summary: str = ""

class StockMarketAgentJupyter:
    def __init__(self, model_name="gpt-4-turbo"):
        """Initialize the agent with a specified language model."""
        self.llm = ChatOpenAI(
            model=model_name, 
            temperature=0.3, 
            max_tokens=1000
        )
    
    async def run_expert_analysis(self, expert_func, state):
        """Async wrapper for expert analysis to handle potential delays."""
        try:
            result = expert_func(state)
            return result
        except Exception as e:
            return {
                "expert": "Error",
                "analysis": f"Analysis failed: {str(e)}"
            }
    
    def financial_analyst(self, state):
        """Expert focused on fundamental financial analysis."""
        prompt = f"""Analyze the stock market question from a financial fundamentals perspective:
        Question: {state.question}
        
        Provide analysis focusing on:
        - Valuation metrics
        - Company/market fundamentals
        - Long-term financial health
        
        Your analysis should be concise, precise, and data-driven."""
        
        response = self.llm.invoke(prompt).content
        return {
            "expert": "Financial Analyst",
            "analysis": response
        }
    
    def news_analyst(self, state):
        """Expert analyzing recent news and sentiment."""
        prompt = f"""Analyze recent news and sentiment related to the market question:
        Question: {state.question}
        
        Provide analysis focusing on:
        - Recent significant news
        - Media sentiment
        - Potential market-moving events
        
        Your analysis should be timely and contextual."""
        
        response = self.llm.invoke(prompt).content
        return {
            "expert": "News Analyst", 
            "analysis": response
        }
    
    def day_trader(self, state):
        """Expert focused on technical analysis and short-term trends."""
        prompt = f"""Perform technical analysis for the market question:
        Question: {state.question}
        
        Provide analysis focusing on:
        - Short-term price patterns
        - Technical indicators
        - Trading volume
        - Potential entry/exit signals
        
        Your analysis should be sharp and actionable."""
        
        response = self.llm.invoke(prompt).content
        return {
            "expert": "Day Trader",
            "analysis": response
        }
    
    def macro_economist(self, state):
        """Expert analyzing broader economic indicators."""
        prompt = f"""Examine the market question through a macroeconomic lens:
        Question: {state.question}
        
        Provide analysis focusing on:
        - Broader economic indicators
        - Global economic trends
        - Monetary and fiscal policy impacts
        
        Your analysis should provide a big-picture perspective."""
        
        response = self.llm.invoke(prompt).content
        return {
            "expert": "Macro Economic Expert",
            "analysis": response
        }
    
    def psychology_expert(self, state):
        """Expert evaluating market sentiment and behavioral aspects."""
        prompt = f"""Analyze market psychology related to the question:
        Question: {state.question}
        
        Provide analysis focusing on:
        - Market sentiment
        - Investor psychology
        - Potential behavioral biases
        - Emotional market drivers
        
        Your analysis should reveal underlying psychological factors."""
        
        response = self.llm.invoke(prompt).content
        return {
            "expert": "Economic Psychology Expert",
            "analysis": response
        }
    
    async def run_analysis(self, question: str):
        """Async method to run the stock market analysis."""
        # Create initial state
        state = StockMarketAgentState(question)
        
        # Define expert functions in order
        expert_funcs = [
            self.financial_analyst,
            self.news_analyst,
            self.day_trader,
            self.macro_economist,
            self.psychology_expert
        ]
        
        # Run analyses sequentially
        for expert_func in expert_funcs:
            analysis = await self.run_expert_analysis(expert_func, state)
            state.expert_analyses.append(analysis)
        
        # Generate consensus
        consensus_prompt = f"""Based on the following expert analyses, generate a concise, one-sentence consensus summary:
        
        {chr(10).join([f"{analysis['expert']}: {analysis['analysis']}" for analysis in state.expert_analyses])}
        
        Synthesize these perspectives into a clear, actionable insight."""
        
        state.consensus_summary = self.llm.invoke(consensus_prompt).content
        
        # Prepare report
        report = {
            "question": state.question,
            "consensus_summary": state.consensus_summary,
            "expert_insights": state.expert_analyses,
            "follow_up_prompt": "Would you like me to elaborate on any part of this analysis?"
        }
        
        return report
    
    def display_report(self, report):
        """Format and display the analysis report."""
        print("=== Stock Market Analysis Report ===")
        print(f"Question: {report['question']}\n")
        print("Consensus Summary:")
        print(report['consensus_summary'], "\n")
        
        print("Expert Insights:")
        for insight in report['expert_insights']:
            print(f"- {insight['expert']}:")
            print(f"  {insight['analysis']}\n")
        
        print("Follow-up:", report['follow_up_prompt'])

# Example usage in Jupyter Notebook
async def main():
    # Ensure you have set OPENAI_API_KEY in your .env file
    agent = StockMarketAgentJupyter()
    
    # Example market question
    market_question = "Do you expect the S&P 500 to go up or down this week?"
    
    # Run the analysis
    report = await agent.run_analysis(market_question)
    
    # Display the report
    agent.display_report(report)

# To run in Jupyter, use:
# await main()